In [9]:
# Import the libraries

import numpy as np
import pandas as pd
import gif
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
from sklearn.metrics import mean_squared_error
import tensorflow as tf
np.random.seed(0)
tf.random.set_seed(0)
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

In [107]:
# Use the helper code below to generate the data

# Defines the number of data points to generate
num_points = 30 

# Generate predictor points (x) between 0 and 5
x = np.linspace(0,5,num_points)

# Generate the response variable (y) using the predictor points
y = x * np.sin(x) + np.random.normal(loc=0, scale=1, size=num_points)

# Generate data of the true function y = x*sin(x) 
# x_b will be used for all predictions below 
x_b = np.linspace(0,5,100)
y_b = x_b*np.sin(x_b)


In [108]:
# Split the data into train and test sets with .33 and random_state = 42
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)


In [112]:
# We will now regularise the NN with L1 regularization
# Initialise the NN, give it a different name for the ease of reading
def make_model():    
    model_2 = Sequential(name='L1regularized')

    n_hidden = 50
    # Add L1 regularization
    myl1_reg = regularizers.l1(0.01) 

    # Add 5 hidden layers with 100 neurons each
    model_2.add(Dense(n_hidden,  kernel_regularizer=myl1_reg, activation='tanh', input_shape=(1,)))
    model_2.add(Dense(n_hidden,  kernel_regularizer=myl1_reg,activation='tanh'))

    # Add the output layer with one neuron 
    model_2.add(Dense(1, kernel_regularizer=myl1_reg, activation='linear'))

    # Compile the model
    model_2.compile(loss='MSE',optimizer=optimizers.Adam(learning_rate=0.01)) 
    return model_2

In [172]:
@gif.frame
def plot_weights(mlp,epochnum=0):
    n_hidden = 50
    weights = {}
    for i in range(0,5,2):
        weights[i] = mlp.get_weights()[i]
    df = pd.DataFrame(columns= ['Layer 1','Layer 2','Layer 3','y'])
    df['Layer 1'] = np.array(list(weights[0].flatten())*n_hidden).reshape(n_hidden,n_hidden).T.reshape(n_hidden**2,)
    df['Layer 2'] = weights[2].flatten()
    df['Layer 3'] = list(weights[4].flatten())*n_hidden
    df.y = 'Weights'
    with plt.xkcd(scale=0.3):
        fig = plt.figure(figsize=(10,6))
        plt.rcParams.update({'font.size': 16})
        numweights = df[(df['Layer 1'].abs() > 0.1) & (df['Layer 2'].abs() > 0.1)].shape[0]
        pd.plotting.parallel_coordinates(df[(df['Layer 1'].abs() > 0.1) & (df['Layer 2'].abs() > 0.1)], "y",
                                         color=["#1C758A"],
                                         cols = ['Layer 1','Layer 2','Layer 3'],
                                         alpha=0.8,lw=3 ) 
        
        plt.title(f'{numweights} non-zero weights after {epochnum} epochs ')

In [181]:
frames = []
model = make_model()
for i in range(10):    
    model.fit(x_train, y_train,  validation_split=0.2, epochs=25, batch_size=10, verbose=0)
    frame = plot_weights(model,epochnum = (i+1)*50)
    frames.append(frame)    

In [186]:
gif.save(frames, 'v3.gif', duration=1.25, unit="s",between = "frames",loop = True)

![](./v3.gif)

![](./v3.gif)